#### Install needed packages

In [ ]:
%pip install --upgrade --quiet langchain langchain-openai langchain_community

#### Check that required API keys are available in environment variables

In [6]:
import os
if not os.environ.get('OPENAI_API_KEY'): print('Warning: OPENAI_API_KEY not found')

#### Maintain session state

In [7]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#### Define simple chain with prompt and LLM

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

LLM_MODEL = 'gpt-4o'

llm = ChatOpenAI(model=LLM_MODEL)

prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('user', '{input}')
])
simple_chat_chain = prompt | llm 
conversational_chain = RunnableWithMessageHistory(
  simple_chat_chain,
  get_session_history,  
  input_messages_key='input',
  history_messages_key='chat_history'
)


#### Content streamer

In [9]:
async def generate_chat_events(input, session_id):
    async for evt in conversational_chain.astream_events(input, version='v1', config={'configurable': {'session_id': session_id}} ):
        if evt['event'] == 'on_chain_stream' and 'seq:step:2' in evt['tags']:
            yield evt['data']['chunk'].content

#### Try it out

2 questions are asked using the same session ID.  Answering the second question requires chat history containing the answer from the first question.

In [15]:
sessionid = 'abc123'
question_1  = 'What is the capital of France?'
question_2  = 'What is its most famous landmark?'

async for event in generate_chat_events({'input': question_1, 'chat_history': []}, sessionid):
    print(event, end='')
print('\n')
    
async for event in generate_chat_events({'input': question_2, 'chat_history': []}, sessionid):
    print(event, end='')

The capital of France is Paris.

The most famous landmark in Paris is the Eiffel Tower.